## Import Data and Libraries 

In [1]:
import pandas as pd
import numpy as np

In [2]:
transactions = pd.read_csv('Dataset/transactions_v2.csv')

In [3]:
transactions.shape

(1431009, 9)

In [4]:
transactions['msno'].unique().shape

(1197050,)

In [5]:
train = pd.read_csv('Dataset/train_v2.csv')

In [6]:
train.shape

(970960, 2)

In [7]:
from sklearn.preprocessing import LabelEncoder #encoder package of sklearn
le = LabelEncoder() #le variable has been assigned a labelencoder function

transactions['msno'] = le.fit_transform(transactions['msno'].astype(str))
train['msno'] = le.fit_transform(train['msno'].astype(str))

In [8]:
trial_join = pd.merge(train, transactions, on='msno', how='inner')

In [9]:
trial_join.shape

(1161173, 10)

In [10]:
df = trial_join

In [11]:
from datetime import datetime, date

In [12]:
df['transaction_date'] = df['transaction_date'].astype(str)      

In [13]:
df['day'] = df['transaction_date'].str.slice(6,8,1)
df['month'] = df['transaction_date'].str.slice(4,6,1)
df['year'] = df['transaction_date'].str.slice(0,4,1)
df['trans_date'] = pd.to_datetime(df[['day', 'month', 'year']])

In [14]:
df=df.drop(['day','year','month'],axis=1)

## Creating New Dataframes for LTV 

In [15]:
import datetime as dft
df_ltv = df[(df.trans_date >= dft.datetime(2017,3,1)) & (df.trans_date < dft.datetime(2017,3,31))].reset_index(drop=True)

In [16]:
df_ltv.shape

(711530, 11)

In [17]:
df_ltv['msno'].value_counts().shape

(682449,)

In [22]:
df_ltv.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_date
0,961455,1,41,30,100,100,1,20170311,20170411,0,2017-03-11
1,162509,1,37,30,149,149,1,20170303,20170405,0,2017-03-03
2,335494,1,38,30,149,149,0,20170303,20170402,0,2017-03-03
3,736830,1,38,30,149,149,0,20170305,20170404,0,2017-03-05
4,771492,1,37,30,149,149,1,20170302,20170402,0,2017-03-02


In [35]:
df_train_ltv = df[(df.trans_date < dft.datetime(2017,2,28))].reset_index(drop=True)

In [36]:
df_train_ltv.shape

(227493, 11)

In [37]:
df_train_ltv['msno'].value_counts().shape

(149120,)

In [38]:
df_train_ltv.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_date
0,891096,1,22,410,1788,1788,0,20160804,20170918,0,2016-08-04
1,652319,1,32,410,1788,1788,0,20160421,20170608,0,2016-04-21
2,867188,1,41,30,99,99,1,20170207,20170702,0,2017-02-07
3,867188,1,41,30,99,99,1,20161107,20170403,0,2016-11-07
4,867188,1,41,30,99,99,1,20170107,20170604,0,2017-01-07


In [40]:
df_ltv_train_just_revenue = df_train_ltv.groupby("msno")["actual_amount_paid"].sum()

In [41]:
df_ltv_train_just_count_subs = df_train_ltv.groupby("msno")["payment_plan_days"].sum()

In [42]:
import datetime as dft
a = dft.datetime(2017,4,1)

In [43]:
df_train_ltv['Days_Diff'] = ( a - df_train_ltv['trans_date']).dt.days

In [44]:
df_ltv_train_just_recency = df_train_ltv.groupby("msno")["Days_Diff"].min()

In [45]:
user_rfm_1 = pd.merge(df_ltv_train_just_revenue, df_ltv_train_just_recency, on='msno', how='inner')

In [46]:
user_rfm = pd.merge(user_rfm_1, df_ltv_train_just_count_subs, on='msno', how='inner')

In [47]:
user_rfm.rename(columns = {'actual_amount_paid':'Monetary', 'Days_Diff':'Recency','payment_plan_days':'Frequency'}, inplace = True)

In [49]:
auto_renew = df_train_ltv.groupby("msno")["is_auto_renew"].max()

In [50]:
user_summary = pd.merge(user_rfm, auto_renew, on='msno', how='inner')

In [51]:
user_summary_for_ltv_train = user_summary

In [52]:
user_summary_for_ltv_train

,Monetary,Recency,Frequency,is_auto_renew
msno,,,,
0,1599,160,395,0
7,1788,271,410,0
23,1788,71,410,0
27,894,186,195,0
35,298,60,90,0
...,...,...,...,...
970938,2094,58,562,0
970941,1788,250,410,0
970942,1788,244,410,0


In [ ]:
df_ltv

In [53]:
df_ltv['discount']=df_ltv['plan_list_price']-df_ltv['actual_amount_paid']

In [54]:
df_ltv

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_date,Days_Diff,discount
0,961455,1,41,30,100,100,1,20170311,20170411,0,2017-03-11,21,0
1,162509,1,37,30,149,149,1,20170303,20170405,0,2017-03-03,29,0
2,335494,1,38,30,149,149,0,20170303,20170402,0,2017-03-03,29,0
3,736830,1,38,30,149,149,0,20170305,20170404,0,2017-03-05,27,0
4,771492,1,37,30,149,149,1,20170302,20170402,0,2017-03-02,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
711525,645624,0,37,30,149,149,1,20170319,20170418,0,2017-03-19,13,0
711526,22156,0,41,30,149,149,1,20170325,20170426,0,2017-03-25,7,0
711527,457614,0,41,30,99,99,1,20170307,20170407,0,2017-03-07,25,0
711528,706976,0,41,30,149,149,1,20170313,20170413,0,2017-03-13,19,0


In [55]:
df_ltv['lastMonthLTV']=df_ltv['actual_amount_paid']-df_ltv['discount']

In [56]:
df_ltv.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,trans_date,Days_Diff,discount,lastMonthLTV
0,961455,1,41,30,100,100,1,20170311,20170411,0,2017-03-11,21,0,100
1,162509,1,37,30,149,149,1,20170303,20170405,0,2017-03-03,29,0,149
2,335494,1,38,30,149,149,0,20170303,20170402,0,2017-03-03,29,0,149
3,736830,1,38,30,149,149,0,20170305,20170404,0,2017-03-05,27,0,149
4,771492,1,37,30,149,149,1,20170302,20170402,0,2017-03-02,30,0,149


In [57]:
is_dis = df_ltv.groupby("msno")["lastMonthLTV"].sum()

In [58]:
is_dis

msno
1          99
3         149
4         149
8         149
9         149
         ... 
970953    149
970954     99
970957    129
970958     99
970959    149
Name: lastMonthLTV, Length: 682449, dtype: int64

In [59]:
user_summary_ltv = pd.merge(user_summary_for_ltv_train, is_dis, on='msno', how='inner')

In [67]:
user_summary_ltv.head()

,Monetary,Recency,Frequency,is_auto_renew,lastMonthLTV
msno,,,,,
99,1788,43,410,0,1200
193,99,416,30,1,99
209,5382,479,990,1,129
236,3078,466,660,1,248
343,180,52,30,1,180


In [68]:
user_summary_ltv.reset_index(level=0, inplace=True)

In [69]:
user_summary_ltv.shape

(12047, 6)

In [70]:
user_summary_ltv.to_csv('TransformedDataset/transformed_user_summary_ltv.csv',index=False)